# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

FileNotFoundError: [Errno 2] No such file or directory: 'data/us_job_market_discrimination.dta'

In [ ]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

In [ ]:
data.head()

<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

In [ ]:
w = data[data.race=='w']
b = data[data.race=='b']
#1. Z test since sample size larger than 30 based on the CLT
#2.null hypothesis: Race doesnt have a significant impact on the rate of callbacks
#alternate hypothesis: Race does have a significant impact on the rate of callbacks

In [ ]:
# Your solution to Q3 here

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

In [ ]:
total_w = w.race.count()
p_w = sum(w[w.race=='w'].call)/w.race.count()
q_w = 1- p_w
#fraction of white sounding names who got a call

In [ ]:
total_b = b.race.count()
p_b = sum(b[b.race=='b'].call)/b.race.count()
q_b = 1 - p_b
#fraction of black sounding names who got a call

In [ ]:
p_hat = (total_w * p_w + total_b * p_b) / (total_b + total_w)
p_hat

In [ ]:
z_val = (p_w - p_b) / np.sqrt(p_hat * (1 - p_hat) * (1/total_w + 1/total_b))
z_critical = stats.norm.ppf(q = 0.975)
z_val

In [ ]:
print(z_critical)
print(p_w)
print(q_w)
print(p_b)
print(q_b)

In [ ]:
#margin of error and confidence interval of .95
upper_interval = (p_w - p_b) + 1.96 * np.sqrt((.096509*.903490/2435) + (.06447*.9355/2435))
lower_interval = (p_w - p_b) - 1.96 * np.sqrt((.096509*.903490/2435) + (.06447*.9355/2435))

In [ ]:
print(upper_interval)
print(lower_interval)

In [ ]:
import scipy
scipy.stats.norm.sf(abs(z_val))*2 

In [ ]:
#result significant reject null hyp

In [ ]:
#Start of a two sample bootstrap test
#define the function 
def bootstrap_replicate_1d(data, func):
    return func(np.random.choice(data, size=len(data)))

In [ ]:
def draw_bs_reps(data, func, size):
    """Draw bootstrap replicates."""

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data, func)

    return bs_replicates

In [ ]:
mean_call = (p_w + p_b) / 2

In [ ]:
mean_call

In [ ]:
p_w_shifted = w.call - np.mean(w.call) + mean_call
p_b_shifted = b.call - np.mean(b.call) + mean_call

In [ ]:
# Compute 10,000 bootstrap replicates from shifted arrays
bs_replicates_a = draw_bs_reps(p_w_shifted, np.mean, size=10000)
bs_replicates_b = draw_bs_reps(p_b_shifted, np.mean, size=10000)

In [ ]:
np.mean(w.call)

In [ ]:
bs_replicates_a

In [ ]:
bs_replicates_b

In [ ]:
empirical_diff_means = p_w - p_b

In [ ]:
bs_replicates = bs_replicates_a - bs_replicates_b

# Compute and print p-value: p
p = np.sum(bs_replicates >= empirical_diff_means) / len(bs_replicates)
print('p-value =', p)

In [ ]:
data.columns

Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?
Ans: No this doesnt mean that race/ name is the most important factor in callback success. There could be many other features such as education level, experience etc that could play a role in determining callback success. We also don't know if the researchers took great care to make sure that the resumes were identical aside from the names. Examining the resumes closely for differences in education and work experience would be helpful to find out how much bias was introduced.